In [1]:
import numpy as np
import pandas as pd

In [ ]:
reg_ratios = pd.read_csv("RegularSeasonDetailedResultsRatios.csv")
tour_ratios = pd.read_csv("NCAATourneyDetailedResultsRatios.csv")

# Get columns
all_cols = reg_ratios.columns.tolist()
cols_to_drop = ["Season", "WTeamID", "WScore", "LTeamID", "LScore", "NumTeamWon", "Loc"]
stats_columns = [item for item in all_cols if item not in cols_to_drop]

# seasons = pd.unique(reg_ratios["Season"])
seasons = [2011, 2012, 2013, 2014, 2015, 2016, 2017]
print(seasons)

In [ ]:
team_ids = pd.DataFrame(index=range(7*68), columns=["Season", "TeamID"])
for i in range(len(seasons)):
    season = seasons[i]
    team_ids_for_season = pd.unique(tour_ratios.loc[tour_ratios["Season"] == season, ["WTeamID", "LTeamID"]].values.ravel('K'))
    for j in range(len(team_ids_for_season)):
        team_ids.loc[i*68+j, "Season"] = season
        team_ids.loc[i*68+j, "TeamID"] = team_ids_for_season[j]
        
# Create a data sets to hold regular season averages for every team in the tournament for every specified year
reg_season_avg_cols = ["Season", "TeamID"] + stats_columns
reg_season_avgs = pd.DataFrame(index=range(team_ids.shape[0]), columns=reg_season_avg_cols)
reg_season_avgs["Season"] = team_ids["Season"]
reg_season_avgs["TeamID"] = team_ids["TeamID"]

for season in seasons:
    team_ids_for_season = reg_season_avgs.loc[reg_season_avgs["Season"] == season, "TeamID"].tolist()
        
    for team_id in team_ids_for_season:
        games_all = reg_ratios.loc[(reg_ratios["WTeamID"] == team_id) | (reg_ratios["LTeamID"] == team_id)]
        games = games_all.loc[games_all["Season"] == season]
        games.reset_index(inplace=True, drop=True)
        games_stats = games.loc[:, stats_columns]
        for i, row in games.iterrows():
            if ((team_id == row["WTeamID"]) & (row["NumTeamWon"] == 0)) | ((team_id == row["LTeamID"]) & (row["NumTeamWon"] == 1)):
                games_stats.iloc[i] = games_stats.iloc[i].apply(np.reciprocal)
    
        reg_season_avgs.loc[(reg_season_avgs["TeamID"] == team_id) & (reg_season_avgs["Season"] == season), stats_columns] = games_stats.mean(axis=0).ravel()

reg_season_avgs.to_csv("RegularSeasonAverageRatios.csv")

# Now, create all regular season averages for all teams, even if not in tournament

In [7]:
root = "/home/austin/Github/kaggle-ncaa-2018/"
reg_ratios = pd.read_csv(root + "derived_data/ratios/RegularSeasonDetailedResultsRatios.csv")

# Get columns
all_cols = reg_ratios.columns.tolist()
cols_to_drop = ["Season", "WTeamID", "WScore", "LTeamID", "LScore", "NumTeamWon", "Loc"]
stats_columns = [item for item in all_cols if item not in cols_to_drop]

# seasons = pd.unique(reg_ratios["Season"])
seasons = range(2003, 2018)

In [8]:
reg_season_avg_cols = ["Season", "TeamID"] + stats_columns
avg_df = pd.DataFrame(columns=reg_season_avg_cols)

for season in seasons:
    team_ids_for_season = pd.unique(reg_ratios.loc[reg_ratios["Season"] == season, ["WTeamID", "LTeamID"]].values.ravel('K'))
    for team_id in team_ids_for_season:
        avg_df = avg_df.append({"Season": season, "TeamID": team_id}, ignore_index=True)
        
        games_all = reg_ratios.loc[(reg_ratios["WTeamID"] == team_id) | (reg_ratios["LTeamID"] == team_id)]
        games = games_all.loc[games_all["Season"] == season]
        games.reset_index(inplace=True, drop=True)
        games_stats = games.loc[:, stats_columns]
        for i, row in games.iterrows():
            if ((team_id == row["WTeamID"]) & (row["NumTeamWon"] == 0)) | ((team_id == row["LTeamID"]) & (row["NumTeamWon"] == 1)):
                games_stats.iloc[i] = games_stats.iloc[i].apply(np.reciprocal)
                
        avg_df.loc[(avg_df["TeamID"] == team_id) & (avg_df["Season"] == season), stats_columns] = games_stats.mean(axis=0).ravel()
        
# avg_df.to_csv(root + "derived_data/ratios/RegularSeasonAverageStats.csv")

In [9]:
reg_comp = pd.read_csv(root + "derived_data/composite/RegularSeasonCompositeStats.csv")

comp_columns = [item for item in reg_comp.columns.tolist() if item not in ["Season", "TeamID"]]

avg_df_test = avg_df

In [10]:
# print(avg_df.shape)
# print(reg_comp.loc[reg_comp["Season"] > 2010].shape)
# print(comp_columns)

# Add columns
for comp_column in comp_columns:
    avg_df_test[comp_column] = float('NaN')
    
for i in range(avg_df.shape[0]):
    season = avg_df_test.loc[i, "Season"]
    team_id = avg_df_test.loc[i, "TeamID"]
    reg_comp_row = reg_comp.loc[(reg_comp["Season"] == season) & (reg_comp["TeamID"] == team_id)]
    
    if(reg_comp_row.empty):
        continue
    reg_comp_row.reset_index(inplace=True, drop=True)
    reg_comp_row = reg_comp_row.iloc[0]
    
    if i == 1:
        print(reg_comp_row[comp_columns])
        print(type(reg_comp_row[comp_columns]))
    
    avg_df_test.loc[(avg_df_test["Season"] == season) & (avg_df_test["TeamID"] == team_id), comp_columns] = reg_comp_row[comp_columns].ravel()

PIE              0.597778
FG_PCT           0.498337
TURNOVER_RATE    0.163772
OFF_REB_PCT      0.366803
FT_RATE          0.396624
4FACTOR          0.373132
OFF_EFF          1.112507
DEF_EFF          0.982265
ASSIST_RATIO     0.164072
DEF_REB_PCT      0.683904
FT_PCT           0.628299
WINPCT           0.793103
Name: 0, dtype: float64
<class 'pandas.core.series.Series'>


In [ ]:
print(avg_df_test)

In [11]:
avg_df_test.to_csv(root + "derived_data/ratios/RegularSeasonAverageRatiosWithComposites.csv")